**The Code in this document tries to predict the 1v1 win chances of players for an age of empires match**

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, brier_score_loss

Step 1: Create the dataframe to work with

In [16]:
df = pd.read_csv('final2.csv')
df.head()

,team_size,team_A_avg_mmr,team_B_avg_mmr,team_A_civs,team_B_civs,map_name,team_A_won,civ_A_vs_B_winrate,civ_A_vs_B_certainty,mmr_gap
0,1,2294.0,2297.0,french,house_of_lancaster,Dry Arabia,0,42.993025,0.503591,-3.0
1,1,2290.0,2165.0,mongols,chinese,Gorge,1,53.023663,0.112949,125.0
2,1,2282.0,2173.0,japanese,chinese,Carmel,1,59.821429,0.135038,109.0
3,1,1958.0,2279.0,ottomans,japanese,Lipany,0,44.265081,0.116866,-321.0
4,1,1632.0,2279.0,holy_roman_empire,english,Dry Arabia,0,51.759944,0.607073,-647.0


In [30]:
X = df.drop(columns=['team_A_won'])
y = df['team_A_won']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = ["team_A_civs", "team_B_civs", "map_name"]
numerical_features = [col for col in X.columns if col not in categorical_features]

In [31]:
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features)
], remainder='passthrough')

model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", HistGradientBoostingClassifier(loss='squared_error'))  # can try 'log_loss' too
])

model.fit(X_train, y_train)

InvalidParameterError: The 'loss' parameter of HistGradientBoostingClassifier must be a str among {'log_loss'} or an instance of 'sklearn._loss.loss.BaseLoss'. Got 'squared_error' instead.

In [27]:
probs = model.predict_proba(X_test)[:, 1] 


print("Log loss:", log_loss(y_test, probs))
print("Brier score:", brier_score_loss(y_test, probs))

AttributeError: This 'Pipeline' has no attribute 'predict_proba'